In [1]:
! pip install soykeyword
! pip install soyclustering
! pip install soynlp # 파이썬에서 한글관련 테스트처리 패키지
! pip install konlpy  # 한글형태소 분석기
! pip install pyLDAvis # 토픽 모델링 시각화
! bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh) # 한글 형태소 분석기
# 오래걸림

     |████████████████████████████████| 419kB 2.8MB/s 
     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 9.9MB/s 
     |████████████████████████████████| 3.6MB 38.2MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 16.7MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=556aacf9f315867b7054d8136f3d8bce877899cfaa522a0ef904393fff21a387
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=3fd7c425b6af8401a941002d654a7c1ce284cc0c49a0b6575a41bb4

In [0]:
import pandas as pd

In [0]:
fpath = '/content/Cat_NewsResult_20200211-20200222.xlsx'
df = pd.read_excel(fpath, index_col=[0])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13620 entries, 0 to 19020
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   뉴스 식별자     13620 non-null  float64
 1   일자         13620 non-null  int64  
 2   언론사        13620 non-null  object 
 3   기고자        12261 non-null  object 
 4   제목         13620 non-null  object 
 5   통합 분류1     13620 non-null  object 
 6   통합 분류2     8475 non-null   object 
 7   통합 분류3     4332 non-null   object 
 8   사건/사고 분류1  1000 non-null   object 
 9   사건/사고 분류2  161 non-null    object 
 10  사건/사고 분류3  57 non-null     object 
 11  인물         7745 non-null   object 
 12  위치         12486 non-null  object 
 13  기관         13382 non-null  object 
 14  키워드        13620 non-null  object 
 15  특성추출       13620 non-null  object 
 16  본문         13620 non-null  object 
 17  URL        13534 non-null  object 
 18  분석제외 여부    818 non-null    object 
 19  category   13620 non-null  object 
dtypes: flo

In [5]:
df.head(1)

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출,본문,URL,분석제외 여부,category
0,1.500601e+06,20200222,매일신문,김교영,"[매일칼럼] 코로나 사태, 우리 함께 이겨내리라!",사회>의료_건강,국제>국제일반,NaN,NaN,NaN,NaN,NaN,"대도,우한,대구,서문시장,청도,대구경북","대구,한국,대구경북","코로나,사태,코로나19,사태,위기,국면,대구경북,초비상,확진자,급증,발생지,확대,감...","대구,대구경북,지역사회,확진자,감염병,공동체,시민들,한국,방역당국,사람들,자가격리,...","코로나19 사태가 위기 국면이다. 특히 대구경북은 초비상이다. 확진자가 급증하고, ...",https://news.imaeil.com/InnerColumn/2020022221...,NaN,social


In [6]:
df.category.value_counts()

intl         4985
social       3443
economics    2591
poltics      1535
culture      1066
Name: category, dtype: int64

# Data preprocessing
- Lab3-Machine-Learning에서 사용했던 코드를 재사용
    -  전처리  코드
    -  TF(IDF) 임베딩 코드

In [0]:
from konlpy.tag import Mecab
from soynlp.normalizer import *

def my_preprocessor(text):
  text = only_hangle_number(text)
  return text

mecab = Mecab()
def my_tokenizer(text):
  return [noun for noun in mecab.nouns(text) if len(noun) > 1]

# Topic Modeling with K-means clustering

In [0]:
docs = df['본문'].values

In [14]:
docs[0][:140]

'코로나19 사태가 위기 국면이다. 특히 대구경북은 초비상이다. 확진자가 급증하고, 발생지는 확대되고 있다. 감염원을 알 수 없는 환자가 늘고 있다. 이제는 내 집 앞에서 확진자가 나오고 있다. 지역사회 감염이 본격화된 것이다. \n \n 대구 도심 거리는'

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizing
vectorizer = TfidfVectorizer(lowercase=False, preprocessor= my_preprocessor, tokenizer= my_tokenizer, max_features=1000)
X = vectorizer.fit_transform(docs)

In [17]:
from soyclustering import SphericalKMeans

# 유클리드가 아니라 코사인시뮬리티? 방식으로 거리계산
spherical_kmeans = SphericalKMeans(n_clusters=80, verbose=1)

# n_cluesters 크게잡고 시작, log를 보여주는 옵션은 verbose인듯
labels = spherical_kmeans.fit_predict(X)

initialization_time=0.078534 sec, sparsity=0.0208
n_iter=1, changed=13405, inertia=9560.502, iter_time=0.254 sec, sparsity=0.507
n_iter=2, changed=4145, inertia=7717.992, iter_time=0.249 sec, sparsity=0.467
n_iter=3, changed=2063, inertia=7403.103, iter_time=0.244 sec, sparsity=0.458
n_iter=4, changed=1206, inertia=7272.162, iter_time=0.250 sec, sparsity=0.455
n_iter=5, changed=794, inertia=7200.130, iter_time=0.249 sec, sparsity=0.454
n_iter=6, changed=519, inertia=7160.621, iter_time=0.243 sec, sparsity=0.453
n_iter=7, changed=370, inertia=7139.478, iter_time=0.249 sec, sparsity=0.452
n_iter=8, changed=324, inertia=7124.861, iter_time=0.262 sec, sparsity=0.452
n_iter=9, changed=293, inertia=7112.304, iter_time=0.242 sec, sparsity=0.451
n_iter=10, changed=243, inertia=7099.178, iter_time=0.241 sec, sparsity=0.451


In [0]:
import numpy as np

def labels_to_doc_topic_prob(labels):
    n_clusters = np.unique(labels).shape[0]
    n_docs = labels.shape[0]
    doc_topic_prob = np.zeros((n_docs, n_clusters))

    for c in range(n_clusters):
        idx = np.where(labels == c)[0]
        doc_topic_prob[idx, c] = 1

    return doc_topic_prob

In [0]:
def labels_x_to_topic_term_prob(labels, x):
    n_clusters = np.unique(labels).shape[0]
    n_terms = x.shape[1]

    topic_term_prob = np.zeros((n_clusters, n_terms))
    for c in range(n_clusters):
        idx = np.where(labels == c)[0]
        topic_term_freq = x[idx].sum(axis=0)
        freq_sum = topic_term_freq.sum()
        if freq_sum == 0:
            continue
        topic_term_prob[c] = topic_term_freq / freq_sum

    base = 1 / n_terms
    rowsum = topic_term_prob.sum(axis=1)
    topic_term_prob[np.where(rowsum == 0)[0]] = base

    return topic_term_prob


In [21]:
import pyLDAvis 

doc_topic_prob = labels_to_doc_topic_prob(labels)
topic_term_prob = labels_x_to_topic_term_prob(labels, X)
doc_lengths = np.asarray(X.sum(axis=1)).reshape(-1)
term_frequency = np.asarray(X.sum(axis=0)).reshape(-1)
idx_to_vocab = {v:k for k, v in vectorizer.vocabulary_.items()}

prepared_data = pyLDAvis.prepare(
    topic_term_prob,
    doc_topic_prob,
    doc_lengths,
    idx_to_vocab,
    term_frequency,
    mds = 'tsne',
    plot_opts = {'xlab': 't-SNE1', 'ylab': 't-SNE2'}
)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)


In [22]:
pyLDAvis.display(prepared_data)

#  연관어 분석

In [24]:
from soykeyword.proportion import CorpusbasedKeywordExtractor

corpusbased_extractor = CorpusbasedKeywordExtractor(
    min_tf=20,
    min_df=2,
    tokenize=my_tokenizer,
    verbose=True
)

# docs: list of str like
corpusbased_extractor.train(docs)

training was done 10333 terms, 13620 docs, memory = 0.457 Gb


In [25]:
keywords = corpusbased_extractor.extract_from_word(
    '마스크',
    min_score = 0.5,
    min_frequency = 10
)

keywords[:20]

[KeywordScore(word='마스크', frequency=1183, score=1.0),
 KeywordScore(word='반출', frequency=20, score=1.0),
 KeywordScore(word='부직포', frequency=23, score=1.0),
 KeywordScore(word='김보성', frequency=22, score=1.0),
 KeywordScore(word='라디오스타', frequency=11, score=1.0),
 KeywordScore(word='에버그린', frequency=15, score=1.0),
 KeywordScore(word='함소원', frequency=18, score=1.0),
 KeywordScore(word='적발', frequency=60, score=0.9971696833765613),
 KeywordScore(word='장갑', frequency=10, score=0.9940436187337552),
 KeywordScore(word='품귀', frequency=61, score=0.9930577802739681),
 KeywordScore(word='무상', frequency=14, score=0.9910919579480315),
 KeywordScore(word='일회용', frequency=19, score=0.9899895993253064),
 KeywordScore(word='매점매석', frequency=31, score=0.9897355570307456),
 KeywordScore(word='착용', frequency=130, score=0.9897169168603807),
 KeywordScore(word='김재룡', frequency=12, score=0.9893293604388242),
 KeywordScore(word='전처', frequency=29, score=0.9888896902117404),
 KeywordScore(word='세정제', frequen



```
Given s (마스크) and w (반출,  부직포,...),
score = p(w|D_s) / { p(w|D_s) + p(w|~D_s)) 
```



#  Keyword similarity
- Jaccard similarity: is defined as the size of the intersection divided by the size of the union of two label sets

In [0]:
keywords_g1 = corpusbased_extractor.extract_from_word(
    '혐오',
    min_score = 0.8,
    min_frequency = 100
)
keywords_g1 = [k.word for k in keywords_g1[:20]]

keywords_g2 = corpusbased_extractor.extract_from_word(
    '외국인',
    min_score = 0.8,
    min_frequency = 100
)
keywords_g2 = [k.word for k in keywords_g2[:20]]

In [28]:
keywords_g1, keywords_g2

(['당부',
  '학생',
  '보호',
  '노력',
  '중국인',
  '영상',
  '독일',
  '교육부',
  '유학생',
  '대학',
  '공포',
  '영국',
  '비판',
  '홈페이지',
  '의혹',
  '이유',
  '인근',
  '거주',
  '중동',
  '전자'],
 ['외국인',
  '코스피',
  '마카오',
  '유학생',
  '검역',
  '관광객',
  '유학',
  '거래일',
  '입국',
  '제한',
  '기숙사',
  '포인트',
  '지수',
  '반도체',
  '연장',
  '대학',
  '마감',
  '교육부',
  '유입',
  '지정'])

In [29]:
intersection = set(keywords_g1).intersection(set(keywords_g2))
union = set(keywords_g1).union(set(keywords_g2))
print('Jaccard', len(intersection) / (len(union) + 0.001))

Jaccard 0.08107888975973623
